# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770695428979                   -0.53    8.0
  2   -2.772135262929       -2.84       -1.30    1.0
  3   -2.772170640394       -4.45       -2.69    2.0
  4   -2.772170722146       -7.09       -4.20    2.0
┌ Warning: Negative ρ detected
│   min_ρ = -2.3636626327597227e-18
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
  5   -2.772170722988       -9.07       -4.73    3.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770772902810                   -0.52    9.0
  2   -2.772060323163       -2.89       -1.32    1.0
  3   -2.772082653729       -4.65       -2.37    2.0
  4   -2.772083417555       -6.12       -3.99    2.0
  5   -2.772083417804       -9.60       -4.98    2.0


1.7736326479621243

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770705434788                   -0.53    8.0
  2   -2.772055698531       -2.87       -1.30    1.0
  3   -2.772082818044       -4.57       -2.62    1.0
  4   -2.772083409331       -6.23       -3.68    2.0
  5   -2.772083416962       -8.12       -4.05    2.0

Polarizability via ForwardDiff:       1.7726228198800071
Polarizability via finite difference: 1.7736326479621243
